In [1]:
import numpy as np
import pandas as pd
import sys
import re
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [2]:
data_path='F:\\study\\ml\\DataSet\\Drug_Molecule\\'
df_protein_train=pd.read_csv('%sdf_protein_train.csv' % (data_path))
df_protein_test=pd.read_csv('%sdf_protein_test.csv' % (data_path))
df_molecule=pd.read_csv('%sdf_molecule.csv'% (data_path))
df_affinity_train=pd.read_csv('%sdf_affinity_train.csv'% (data_path))
df_affinity_test=pd.read_csv('%sdf_affinity_test.csv'% (data_path))

In [3]:
df_affinity_train.Ki.describe()

count    165084.000000
mean          7.676823
std           1.567519
min          -4.000000
25%           6.543634
50%           7.698970
75%           8.823909
max          16.000000
Name: Ki, dtype: float64

In [4]:
df_affinity_test['Ki']=-11
data=pd.concat([df_affinity_train,df_affinity_test])

In [5]:
protein_concat = pd.concat([df_protein_train,df_protein_test])

In [6]:
df_molecule.head()

,Molecule_ID,Fingerprint,cyp_3a4,cyp_2c9,cyp_2d6,ames_toxicity,fathead_minnow_toxicity,tetrahymena_pyriformis_toxicity,honey_bee,cell_permeability,logP,renal_organic_cation_transporter,CLtotal,hia,biodegradation,Vdd,p_glycoprotein_inhibition,NOAEL,solubility,bbb
0,0,"1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0...",0.311594,0.742983,0.363031,0.623255,0.005905,NaN,0.800019,4.919215,NaN,0.479723,3.940952,0.029493,0.003006,24.211564,0.260743,-0.757354,-5.401525,0.087650
1,1,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",NaN,0.854373,0.280685,NaN,0.034951,2.174478,0.846931,4.723896,4.318473,0.492733,10.394106,NaN,0.016205,27.109723,0.511252,NaN,-5.089023,0.000608
2,2,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.102385,0.853763,0.197318,0.745154,0.030430,2.038387,0.854921,4.731113,NaN,0.403998,8.909931,0.011798,0.020116,20.560356,0.439826,-1.163596,-5.124717,0.000570
3,3,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.315907,0.855956,0.376436,NaN,0.033857,NaN,NaN,4.470623,NaN,0.699078,6.572888,0.010473,0.019070,14.791946,0.604665,-1.155990,-4.973178,0.029107
4,4,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.106763,0.844784,0.207776,NaN,NaN,1.895097,0.859958,4.723733,4.193617,0.438007,9.628845,0.011194,0.018409,22.261876,0.494917,-1.158525,-4.886913,0.000603


In [7]:
feat=pd.DataFrame(df_molecule.Fingerprint.apply(lambda x:x.split(',')).tolist())

In [8]:
feat.head()

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,1,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,1
1,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
3,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
4,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [9]:
feat.columns=['FP_{}'.format(i) for i in range(feat.shape[1])]
feat=feat.astype('int')

In [10]:
feat.head()

,FP_0,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,...,FP_157,FP_158,FP_159,FP_160,FP_161,FP_162,FP_163,FP_164,FP_165,FP_166
0,1,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,1,1
1,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
3,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
4,1,1,0,1,0,1,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [11]:
data.head()

,Protein_ID,Molecule_ID,Ki
0,0,0,8.309804
1,1,1,10.292430
2,2,1,6.778586
3,1,2,10.769551
4,2,2,6.170568


In [12]:
feat['Molecule_ID']=df_molecule.Molecule_ID

In [13]:
feat.head()

,FP_0,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,FP_7,FP_8,FP_9,...,FP_158,FP_159,FP_160,FP_161,FP_162,FP_163,FP_164,FP_165,FP_166,Molecule_ID
0,1,0,0,1,0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,1,0
1,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,2
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,3
4,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,4


In [14]:
data=data.merge(feat,on='Molecule_ID',how='left')

In [15]:
df_molecule.head()

,Molecule_ID,Fingerprint,cyp_3a4,cyp_2c9,cyp_2d6,ames_toxicity,fathead_minnow_toxicity,tetrahymena_pyriformis_toxicity,honey_bee,cell_permeability,logP,renal_organic_cation_transporter,CLtotal,hia,biodegradation,Vdd,p_glycoprotein_inhibition,NOAEL,solubility,bbb
0,0,"1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0...",0.311594,0.742983,0.363031,0.623255,0.005905,NaN,0.800019,4.919215,NaN,0.479723,3.940952,0.029493,0.003006,24.211564,0.260743,-0.757354,-5.401525,0.087650
1,1,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",NaN,0.854373,0.280685,NaN,0.034951,2.174478,0.846931,4.723896,4.318473,0.492733,10.394106,NaN,0.016205,27.109723,0.511252,NaN,-5.089023,0.000608
2,2,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.102385,0.853763,0.197318,0.745154,0.030430,2.038387,0.854921,4.731113,NaN,0.403998,8.909931,0.011798,0.020116,20.560356,0.439826,-1.163596,-5.124717,0.000570
3,3,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.315907,0.855956,0.376436,NaN,0.033857,NaN,NaN,4.470623,NaN,0.699078,6.572888,0.010473,0.019070,14.791946,0.604665,-1.155990,-4.973178,0.029107
4,4,"1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.106763,0.844784,0.207776,NaN,NaN,1.895097,0.859958,4.723733,4.193617,0.438007,9.628845,0.011194,0.018409,22.261876,0.494917,-1.158525,-4.886913,0.000603


In [16]:
data=data.merge(df_molecule,on='Molecule_ID',how='left')
del data['Fingerprint']

In [17]:
data.head()

,Protein_ID,Molecule_ID,Ki,FP_0,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,...,logP,renal_organic_cation_transporter,CLtotal,hia,biodegradation,Vdd,p_glycoprotein_inhibition,NOAEL,solubility,bbb
0,0,0,8.309804,1,0,0,1,0,1,0,...,NaN,0.479723,3.940952,0.029493,0.003006,24.211564,0.260743,-0.757354,-5.401525,0.087650
1,1,1,10.292430,1,1,0,1,0,1,0,...,4.318473,0.492733,10.394106,NaN,0.016205,27.109723,0.511252,NaN,-5.089023,0.000608
2,2,1,6.778586,1,1,0,1,0,1,0,...,4.318473,0.492733,10.394106,NaN,0.016205,27.109723,0.511252,NaN,-5.089023,0.000608
3,1,2,10.769551,1,1,0,1,0,1,0,...,NaN,0.403998,8.909931,0.011798,0.020116,20.560356,0.439826,-1.163596,-5.124717,0.000570
4,2,2,6.170568,1,1,0,1,0,1,0,...,NaN,0.403998,8.909931,0.011798,0.020116,20.560356,0.439826,-1.163596,-5.124717,0.000570


In [18]:
protein_concat.head()

,Protein_ID,Sequence
0,0,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...
1,1,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...
2,2,MDPLNLSWYDDDLERQNWSRPFNGSEGKADRPHYNYYAMLLTLLIF...
3,3,MDSSTGPGNTSDCSDPLAQASCSPAPGSWLNLSHVDGNQSDPCGLN...
4,4,MEPVPSARAELQFSLLANVSDTFPSAFPSASANASGSPGARSASSL...


In [19]:
texts=[[ word for word in re.findall(r'.{3}',line)]
      for line in (protein_concat.Sequence)]

In [20]:
from  gensim.models import Word2Vec

D:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [21]:
model=Word2Vec(texts,window=8,size=256,sg=1,hs=1,min_count=1,negative=5,sample=0.001,workers=4)

In [22]:
vectors=pd.DataFrame([model[word] for word in model.wv.vocab])

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [23]:
vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-0.153556,0.019382,-0.042481,-0.041756,0.020416,-0.200338,-0.154690,0.178530,-0.030515,0.213012,...,-0.102092,0.062944,0.177592,0.028306,0.005361,0.016141,0.012738,-0.086006,0.040092,0.098001
1,-0.003607,-0.030663,0.037749,0.110384,-0.038136,0.260962,0.181559,0.158313,-0.200388,0.046216,...,-0.237512,-0.256137,0.255282,-0.116975,-0.045082,0.036849,0.019019,-0.035103,-0.033015,0.022028
2,-0.038593,-0.141066,-0.071715,-0.047531,0.002724,-0.281115,-0.079546,0.103657,-0.031453,-0.065310,...,-0.276528,0.138337,0.157874,-0.115533,-0.045102,0.098607,0.088104,-0.016798,0.110131,-0.070661
3,-0.143741,0.023790,-0.343794,-0.358643,0.106314,-0.232175,-0.050014,0.112990,-0.095127,0.027919,...,-0.015023,-0.076710,-0.039496,-0.004838,0.038366,0.159166,0.140615,-0.192309,0.072612,0.037266
4,0.001233,0.253997,0.067822,-0.201269,0.137867,-0.175665,-0.133953,0.083618,-0.100661,0.009825,...,-0.350147,-0.251605,-0.033140,-0.132598,0.238415,0.160724,0.013483,-0.042603,0.089099,0.126147


In [24]:
vectors.columns=['vec_{}'.format(i) for i in range(vectors.shape[1])]

In [25]:
vectors['Word']=model.wv.vocab

In [26]:
vectors.head()

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255,Word
0,-0.153556,0.019382,-0.042481,-0.041756,0.020416,-0.200338,-0.154690,0.178530,-0.030515,0.213012,...,0.062944,0.177592,0.028306,0.005361,0.016141,0.012738,-0.086006,0.040092,0.098001,MDN
1,-0.003607,-0.030663,0.037749,0.110384,-0.038136,0.260962,0.181559,0.158313,-0.200388,0.046216,...,-0.256137,0.255282,-0.116975,-0.045082,0.036849,0.019019,-0.035103,-0.033015,0.022028,VLP
2,-0.038593,-0.141066,-0.071715,-0.047531,0.002724,-0.281115,-0.079546,0.103657,-0.031453,-0.065310,...,0.138337,0.157874,-0.115533,-0.045102,0.098607,0.088104,-0.016798,0.110131,-0.070661,VDS
3,-0.143741,0.023790,-0.343794,-0.358643,0.106314,-0.232175,-0.050014,0.112990,-0.095127,0.027919,...,-0.076710,-0.039496,-0.004838,0.038366,0.159166,0.140615,-0.192309,0.072612,0.037266,DLS
4,0.001233,0.253997,0.067822,-0.201269,0.137867,-0.175665,-0.133953,0.083618,-0.100661,0.009825,...,-0.251605,-0.033140,-0.132598,0.238415,0.160724,0.013483,-0.042603,0.089099,0.126147,PNI


In [27]:
df_word=pd.DataFrame()

In [28]:
word_col=[]
for i in range(len(texts)):
    word_col_line=[]
    for w in range(len(texts[i])):
        word_col_line.append(texts[i][w])
    word_col.extend(word_col_line)
df_word['Word']=word_col

In [29]:
word_id=[]
id_list=list(protein_concat.Protein_ID)
for i in range(len(texts)):
    word_id_line=[]
    for w in range(len(texts[i])):
        word_id_line.append(id_list[i])
    word_id.extend(word_id_line)
df_word['Protein_ID']=word_id

In [30]:
df_word.head()

,Word,Protein_ID
0,MDN,0
1,VLP,0
2,VDS,0
3,DLS,0
4,PNI,0


In [31]:
df_word=df_word.merge(vectors,on='Word',how='left')

In [32]:
df_word.head()

,Word,Protein_ID,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_246,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255
0,MDN,0,-0.153556,0.019382,-0.042481,-0.041756,0.020416,-0.200338,-0.154690,0.178530,...,-0.102092,0.062944,0.177592,0.028306,0.005361,0.016141,0.012738,-0.086006,0.040092,0.098001
1,VLP,0,-0.003607,-0.030663,0.037749,0.110384,-0.038136,0.260962,0.181559,0.158313,...,-0.237512,-0.256137,0.255282,-0.116975,-0.045082,0.036849,0.019019,-0.035103,-0.033015,0.022028
2,VDS,0,-0.038593,-0.141066,-0.071715,-0.047531,0.002724,-0.281115,-0.079546,0.103657,...,-0.276528,0.138337,0.157874,-0.115533,-0.045102,0.098607,0.088104,-0.016798,0.110131,-0.070661
3,DLS,0,-0.143741,0.023790,-0.343794,-0.358643,0.106314,-0.232175,-0.050014,0.112990,...,-0.015023,-0.076710,-0.039496,-0.004838,0.038366,0.159166,0.140615,-0.192309,0.072612,0.037266
4,PNI,0,0.001233,0.253997,0.067822,-0.201269,0.137867,-0.175665,-0.133953,0.083618,...,-0.350147,-0.251605,-0.033140,-0.132598,0.238415,0.160724,0.013483,-0.042603,0.089099,0.126147


In [33]:
del df_word['Word']

In [34]:
df_word=df_word.groupby(['Protein_ID'],as_index=False).agg('mean')

In [35]:
df_word.head()

,Protein_ID,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_246,vec_247,vec_248,vec_249,vec_250,vec_251,vec_252,vec_253,vec_254,vec_255
0,0,-0.061737,0.001474,-0.068388,-0.083552,0.077786,-0.084868,0.001301,0.111345,-0.029593,...,-0.142281,-0.009787,0.168535,0.002609,-0.005595,0.072652,0.092014,-0.092801,0.123956,0.042668
1,1,-0.041925,0.052228,-0.072538,-0.078719,0.075758,-0.084566,0.036415,0.100907,0.004633,...,-0.105568,-0.047862,0.147216,0.057867,0.002743,0.065785,0.039584,-0.119679,0.111326,0.054088
2,2,-0.037502,0.002977,-0.077908,-0.102348,0.082796,-0.108575,-0.005918,0.070792,0.028182,...,-0.089275,-0.037909,0.180872,0.033456,0.009215,0.102726,0.082502,-0.081259,0.049278,0.041818
3,3,-0.030399,0.028006,-0.106582,-0.015764,0.048236,-0.124687,0.009544,0.062172,-0.001528,...,-0.026420,0.006667,0.156417,-0.034498,0.004060,0.104261,0.069500,-0.070515,0.085057,0.056139
4,4,-0.028268,-0.043098,-0.102546,-0.095127,0.080439,-0.108084,-0.015511,0.115721,-0.045792,...,-0.134595,0.015648,0.127162,-0.005345,0.032678,0.079475,0.111862,-0.089772,0.125962,0.046711


In [36]:
df_word.columns=['Protein_ID']+['vec_m_{}'.format(i) for i in range(df_word.shape[1]-1)]

In [37]:
df_word.head()

,Protein_ID,vec_m_0,vec_m_1,vec_m_2,vec_m_3,vec_m_4,vec_m_5,vec_m_6,vec_m_7,vec_m_8,...,vec_m_246,vec_m_247,vec_m_248,vec_m_249,vec_m_250,vec_m_251,vec_m_252,vec_m_253,vec_m_254,vec_m_255
0,0,-0.061737,0.001474,-0.068388,-0.083552,0.077786,-0.084868,0.001301,0.111345,-0.029593,...,-0.142281,-0.009787,0.168535,0.002609,-0.005595,0.072652,0.092014,-0.092801,0.123956,0.042668
1,1,-0.041925,0.052228,-0.072538,-0.078719,0.075758,-0.084566,0.036415,0.100907,0.004633,...,-0.105568,-0.047862,0.147216,0.057867,0.002743,0.065785,0.039584,-0.119679,0.111326,0.054088
2,2,-0.037502,0.002977,-0.077908,-0.102348,0.082796,-0.108575,-0.005918,0.070792,0.028182,...,-0.089275,-0.037909,0.180872,0.033456,0.009215,0.102726,0.082502,-0.081259,0.049278,0.041818
3,3,-0.030399,0.028006,-0.106582,-0.015764,0.048236,-0.124687,0.009544,0.062172,-0.001528,...,-0.026420,0.006667,0.156417,-0.034498,0.004060,0.104261,0.069500,-0.070515,0.085057,0.056139
4,4,-0.028268,-0.043098,-0.102546,-0.095127,0.080439,-0.108084,-0.015511,0.115721,-0.045792,...,-0.134595,0.015648,0.127162,-0.005345,0.032678,0.079475,0.111862,-0.089772,0.125962,0.046711


In [38]:
data.head()

,Protein_ID,Molecule_ID,Ki,FP_0,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,...,logP,renal_organic_cation_transporter,CLtotal,hia,biodegradation,Vdd,p_glycoprotein_inhibition,NOAEL,solubility,bbb
0,0,0,8.309804,1,0,0,1,0,1,0,...,NaN,0.479723,3.940952,0.029493,0.003006,24.211564,0.260743,-0.757354,-5.401525,0.087650
1,1,1,10.292430,1,1,0,1,0,1,0,...,4.318473,0.492733,10.394106,NaN,0.016205,27.109723,0.511252,NaN,-5.089023,0.000608
2,2,1,6.778586,1,1,0,1,0,1,0,...,4.318473,0.492733,10.394106,NaN,0.016205,27.109723,0.511252,NaN,-5.089023,0.000608
3,1,2,10.769551,1,1,0,1,0,1,0,...,NaN,0.403998,8.909931,0.011798,0.020116,20.560356,0.439826,-1.163596,-5.124717,0.000570
4,2,2,6.170568,1,1,0,1,0,1,0,...,NaN,0.403998,8.909931,0.011798,0.020116,20.560356,0.439826,-1.163596,-5.124717,0.000570


In [39]:
data=data.merge(df_word,on='Protein_ID',how='left')

In [40]:
data.head()

,Protein_ID,Molecule_ID,Ki,FP_0,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,...,vec_m_246,vec_m_247,vec_m_248,vec_m_249,vec_m_250,vec_m_251,vec_m_252,vec_m_253,vec_m_254,vec_m_255
0,0,0,8.309804,1,0,0,1,0,1,0,...,-0.142281,-0.009787,0.168535,0.002609,-0.005595,0.072652,0.092014,-0.092801,0.123956,0.042668
1,1,1,10.292430,1,1,0,1,0,1,0,...,-0.105568,-0.047862,0.147216,0.057867,0.002743,0.065785,0.039584,-0.119679,0.111326,0.054088
2,2,1,6.778586,1,1,0,1,0,1,0,...,-0.089275,-0.037909,0.180872,0.033456,0.009215,0.102726,0.082502,-0.081259,0.049278,0.041818
3,1,2,10.769551,1,1,0,1,0,1,0,...,-0.105568,-0.047862,0.147216,0.057867,0.002743,0.065785,0.039584,-0.119679,0.111326,0.054088
4,2,2,6.170568,1,1,0,1,0,1,0,...,-0.089275,-0.037909,0.180872,0.033456,0.009215,0.102726,0.082502,-0.081259,0.049278,0.041818


In [41]:
train_feat=data[data['Ki'] > -11 ].fillna(-999)
test_feat=data[data['Ki'] <= -11].fillna(-999)
label_train=train_feat['Ki']
label_test=test_feat['Ki']

In [42]:

train_feat.shape

(165084, 444)

In [43]:
test_feat.shape

(41383, 444)

In [44]:
train_feat.head()

,Protein_ID,Molecule_ID,Ki,FP_0,FP_1,FP_2,FP_3,FP_4,FP_5,FP_6,...,vec_m_246,vec_m_247,vec_m_248,vec_m_249,vec_m_250,vec_m_251,vec_m_252,vec_m_253,vec_m_254,vec_m_255
0,0,0,8.309804,1,0,0,1,0,1,0,...,-0.142281,-0.009787,0.168535,0.002609,-0.005595,0.072652,0.092014,-0.092801,0.123956,0.042668
1,1,1,10.292430,1,1,0,1,0,1,0,...,-0.105568,-0.047862,0.147216,0.057867,0.002743,0.065785,0.039584,-0.119679,0.111326,0.054088
2,2,1,6.778586,1,1,0,1,0,1,0,...,-0.089275,-0.037909,0.180872,0.033456,0.009215,0.102726,0.082502,-0.081259,0.049278,0.041818
3,1,2,10.769551,1,1,0,1,0,1,0,...,-0.105568,-0.047862,0.147216,0.057867,0.002743,0.065785,0.039584,-0.119679,0.111326,0.054088
4,2,2,6.170568,1,1,0,1,0,1,0,...,-0.089275,-0.037909,0.180872,0.033456,0.009215,0.102726,0.082502,-0.081259,0.049278,0.041818


In [45]:
train_feat.drop(['Protein_ID','Molecule_ID','Ki'],axis=1,inplace=True)
test_feat.drop(['Protein_ID','Molecule_ID','Ki'],axis=1,inplace=True)

In [46]:
params={
    'boosting_type':'gbdt',
    'objective':'rmse',
    'metric':'l2',
    'min_child_weigth':3,
    'num_leaves':2**5,
    'lambda_l2':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'colsample_bylevel':0.7,
    'learning_rate':0.05,
    'tree_method':'exact',
    'seed':2017,
    'nthread':4,
    'silent':True
}

In [47]:
import time

In [ ]:
train=lgb.Dataset(train_feat,label_train)
test=lgb.Dataset(test_feat,label=label_test,reference=train)

In [ ]:
%%time
n=3000
gbm=lgb.train(params=params,
             train_set=train,
             num_boost_round=n,
             verbose_eval=50,
             valid_sets=[train,test])
#0 training's l2: 0.523378	valid_1's l2: 334.936

D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:661: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


[50]	training's l2: 1.69899	valid_1's l2: 344.122
[100]	training's l2: 1.4933	valid_1's l2: 342.089
[150]	training's l2: 1.38538	valid_1's l2: 341.629
[200]	training's l2: 1.30893	valid_1's l2: 340.981
[250]	training's l2: 1.24911	valid_1's l2: 340.468
[300]	training's l2: 1.20071	valid_1's l2: 340.328
[350]	training's l2: 1.15679	valid_1's l2: 340.074
[400]	training's l2: 1.11963	valid_1's l2: 339.774


In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
pred_train=gbm.predict(train_feat)

In [ ]:
np.sqrt(mean_squared_error(label_train,pred_train))  #0.7216673438620845 